In [1]:
import io
import pandas as pd
from pathlib import Path
from pandas_plots import tbl

dir_data=Path("../data")
file_csv="RegisterZurErfassung_DATA_LABELS_2024-01-11_1301.csv"
file_xls="Register_fertig.xlsx"

df_xls=pd.read_excel(dir_data/file_xls, sheet_name="Tabelle fertig")

In [2]:
# * try reading csv file
# _delimiter = "|"
with open(dir_data/file_csv, encoding="windows-1252") as f:
    csv=f.read()

In [3]:
# Replace commas inside double quotes with semicolons
def replace_delimiter_outside_quotes(
    input: str, delimiter_old: str = ",", delimiter_new: str = ";", quotechar: str = '"'
):
    outside_quotes = True
    output = ""
    for char in input:
        if char == quotechar:
            outside_quotes = not outside_quotes
        elif outside_quotes and char == delimiter_old:
            char = delimiter_new
        output += char
    return output

replace_delimiter_outside_quotes(
    'Participant ID,"Survey Identifier","Survey Timestamp",Datum,CTCAE-Bewertung,"Was war das AE? (Schmerz, Schürfwunde etc.)"'
)

'Participant ID;"Survey Identifier";"Survey Timestamp";Datum;CTCAE-Bewertung;"Was war das AE? (Schmerz, Schürfwunde etc.)"'

In [21]:
# Replace commas inside double quotes with semicolons
def reduce_quotes_in_string(input: str):
    output = ""
    len_=len(input)
    i = 0
    while i < len_:
        # * 
        if input[i] == '"' and i+1 < len_ and input[i+1] != '"':
            output += ''
            i = i+1
        elif input[i] == '"'and i+1 < len_ and input[i+1] == '"':
            output+= '"'
            i = i+2
        else:
            output += input[i]
            i=i+1

    return output

lol=reduce_quotes_in_string('4,,""2022-11-08 09:19:23"",,1,""Schmerzen im Hüft- und Kniebereich (kurzzeitig)"",,,No,,Ja,Checked,Unchecked,Unchecked,Unchecked,mild,""weil der Junge nicht geweint hat"",No,No,,No,No,No,,No,,,No,,,,,,,,Checked,Unchecked,Checked,Unchecked,Unchecked,,No,No,,""max. 3 Stunden"",""Luftballonspiel im Stehen"	 Kind hat sich gestreckt und dabei das Gleichgewicht verloren	" Fall auf Seite"",No,Yes,""unter 5 Minuten"",Yes,""Kind hat Anweisung bekommen sich nicht zu strecken und der Ballon wurde vom Therapeuten tiefer gespielt"",Yes,Unchecked,Checked,Unchecked,Yes,Yes,Yes,Akuttherapie,Einzel,""6 bis 9 Jahre"",No,No,Klinikflur,,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,""1. Hälfe"",Checked,Unchecked,Unchecked,Unchecked,Unchecked,,Essen,,Complete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,Incomplete')
lol2=reduce_quotes_in_string('5,,""2022-11-08 09:31:05"",,2,""Schwindel und Fieber"",,Nein,No,,Nein,Unchecked,Unchecked,Unchecked,Unchecked,,,No,Yes,,No,No,No,,Yes,""Gabe von fiebersenkenden Medikamenten."",,No,,,,Nein,,Ja,,Checked,Unchecked,Unchecked,Unchecked,Unchecked,,No,No,,""max. 3 Stunden"",""Beim leichter Mobilisation und Aktivierung hat das Kind Schwindel rückgemeldet"	" es wurde Fieber gemessen und Fieber festgestellt"",No,No,,Yes,,No,Checked,Unchecked,Unchecked,No,Yes,No,Akuttherapie,Einzel,""10 bis 14 Jahre"",No,No,Pat.-Zimmer,,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,""2. Hälfte"",Unchecked,Unchecked,Checked,Unchecked,Checked,""red. AZ"",Essen,,Complete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,Incomplete')
lol3=reduce_quotes_in_string('43,,,2023_10_10,,""Schmerzen ain der Glutealmuskulatur und im Steiß"",,""Weiß nicht"",No,,Ja,Unchecked,Unchecked,Unchecked,Unchecked,,,No,No,,No,No,No,,No,,,No,,,,Nein,,Nein,,Unchecked,Unchecked,Unchecked,Unchecked,Checked,,No,,,,""Betroffener wollte sich aufrichten, um Spazieren zu gehen"	 bevor er sich aufgerichtet hatte, kam es zu einer Schmerz-Attacke	 Arzt dazugerufen	 dieser hat ihn untersucht und Bewegungen in verschiedene Richtungen untersucht	 alles war uauffällig	 Pat. hatte das seit der letzten LP bereits 2 mal	" Bruder von ihn, hatte ihn zuvor am Rücken massiert"",Yes,,,,,Yes,Checked,Checked,Unchecked,No,Yes,Yes,Akuttherapie,Einzel,""15 bis 18 Jahre"",No,No,Pat.-Zimmer,,Unchecked,Unchecked,Checked,Checked,Unchecked,Unchecked,""1. Hälfe"",Unchecked,Unchecked,Checked,Unchecked,Checked,""nach KMT"	 hatte stark abgenommen	" schlechter AZ!"",Essen,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,Incomplete')
lol3

'43,,,2023_10_10,,"Schmerzen ain der Glutealmuskulatur und im Steiß",,"Weiß nicht",No,,Ja,Unchecked,Unchecked,Unchecked,Unchecked,,,No,No,,No,No,No,,No,,,No,,,,Nein,,Nein,,Unchecked,Unchecked,Unchecked,Unchecked,Checked,,No,,,,"Betroffener wollte sich aufrichten, um Spazieren zu gehen\t bevor er sich aufgerichtet hatte, kam es zu einer Schmerz-Attacke\t Arzt dazugerufen\t dieser hat ihn untersucht und Bewegungen in verschiedene Richtungen untersucht\t alles war uauffällig\t Pat. hatte das seit der letzten LP bereits 2 mal\t Bruder von ihn, hatte ihn zuvor am Rücken massiert",Yes,,,,,Yes,Checked,Checked,Unchecked,No,Yes,Yes,Akuttherapie,Einzel,"15 bis 18 Jahre",No,No,Pat.-Zimmer,,Unchecked,Unchecked,Checked,Checked,Unchecked,Unchecked,"1. Hälfe",Unchecked,Unchecked,Checked,Unchecked,Checked,"nach KMT\t hatte stark abgenommen\t schlechter AZ!",Essen,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchec

In [27]:
# csv = csv.replace("\t", "")
l = csv.splitlines()
l2 = []
for line in l[:]:
    if line[:1] == '"':
        line = line[1:-1]
    line=reduce_quotes_in_string(line)
    line=replace_delimiter_outside_quotes(line)
    line=line.replace('"', '')
    l2.append(line)
    print(f'{line.count(";")} {line}')
s = "\n".join(l2).replace('\t', '')
s
df_csv = pd.read_csv(
    io.StringIO(s),
    sep=";",
    encoding="windows-1252",
)
df_csv

188 Participant ID;Survey Identifier;Survey Timestamp;Datum;CTCAE-Bewertung;Was war das AE? (Schmerz, Schürfwunde etc.);Hat jemand mit Expertise  die Situation geprüft?;Diese Person hat ein 'OK' zum Weitermachen gegeben.  ;Wurde etwas Strukturelles verändert?;Was?;Sind Schmerzen aufgetreten?;Wie lange dauerten diese an? (choice=Schmerzen am gleichen Tag);Wie lange dauerten diese an? (choice=Schmerzen bis zum Folgetag);Wie lange dauerten diese an? (choice=Schmerzen, die mindestens 3 Tage anhielten);Wie lange dauerten diese an? (choice=weiß nicht);Wie stark waren diese Schmerzen?;Wie kommen Sie zu dieser Einschätzung?;Kam es zu einem Krankenhausaufenthalt oder einer Verlängerung eines solchen?;Kam es zu einer medizinischen Weiterbehandlung? ;Welche und was wurde gemacht?;Kam es zu einer deutlichen Verzögerung bei der Einhaltung des medizinischen Therapieprotokolls?;Kam es zu lebensnotwendigen Interventionen (Beatmung, Herzdruckmassage etc.)?;Kam es zu einem erhöhten Pflegebedarf;Wie lang

,Participant ID,Survey Identifier,Survey Timestamp,Datum,CTCAE-Bewertung,"Was war das AE? (Schmerz, Schürfwunde etc.)",Hat jemand mit Expertise die Situation geprüft?,Diese Person hat ein 'OK' zum Weitermachen gegeben.,Wurde etwas Strukturelles verändert?,Was?,...,Welche motorische Hauptform dominierte die Bewegungseinheit ? (choice=Schnelligkeit).2,In welcher Phase der geplanten Sporteinheit trat das AE auf?.2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Gut).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Durchschnittlich).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Mäßig).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Weiß nicht).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Anmerkung).2,"Glaubst Du, dass das AE mit dem Sport assoziiert ist?",Anmerkung.2,Complete?.2
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
1,2,NaN,2022-10-13 12:26:32,NaN,1.0,Schürfwunde an der Wange,NaN,Ja,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
2,3,NaN,2022-10-13 13:41:16,NaN,2.0,strake Übelkeit,NaN,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
3,4,NaN,2022-11-08 09:19:23,NaN,1.0,Schmerzen im Hüft- und Kniebereich (kurzzeitig),NaN,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
4,5,NaN,2022-11-08 09:31:05,NaN,2.0,Schwindel und Fieber,NaN,Nein,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
5,6,NaN,2022-11-18 10:10:55,NaN,1.0,Erbrechen,NaN,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
6,7,NaN,2022-11-21 14:02:25,NaN,3.0,Bruch re. oberere Srunggelenk Tibiakopf,NaN,Nein,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
7,8,NaN,2022-12-29 09:24:07,NaN,3.0,Schmerzen und Verspannungen,NaN,NaN,Yes,"Abfrage bei Ärtze-Team, wenn Schmerzen vorher ...",...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
8,9,NaN,2023-01-10 14:35:03,NaN,2.0,plötzliche Übelkeit und Erbrechen während der ...,NaN,Ja,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
9,10,NaN,2023-01-10 14:45:19,NaN,2.0,steckende Rückenschmerzen nach Abrutschen von ...,NaN,NaN,No,NaN,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete


In [35]:
tbl.describe_df(df_csv.iloc[:,:], "df_csv", top_n_uniques=5, top_n_chars_in_index=5)

🔵 *** df: df_csv ***
🟣 shape: (50, 189) columns: ['Participant ID', 'Survey Identifier', 'Survey Timestamp', 'Datum', 'CTCAE-Bewertung', 'Was war das AE? (Schmerz, Schürfwunde etc.)', 'Hat jemand mit Expertise  die Situation geprüft?', "Diese Person hat ein 'OK' zum Weitermachen gegeben.  ", 'Wurde etwas Strukturelles verändert?', 'Was?', 'Sind Schmerzen aufgetreten?', 'Wie lange dauerten diese an? (choice=Schmerzen am gleichen Tag)', 'Wie lange dauerten diese an? (choice=Schmerzen bis zum Folgetag)', 'Wie lange dauerten diese an? (choice=Schmerzen, die mindestens 3 Tage anhielten)', 'Wie lange dauerten diese an? (choice=weiß nicht)', 'Wie stark waren diese Schmerzen?', 'Wie kommen Sie zu dieser Einschätzung?', 'Kam es zu einem Krankenhausaufenthalt oder einer Verlängerung eines solchen?', 'Kam es zu einer medizinischen Weiterbehandlung? ', 'Welche und was wurde gemacht?', 'Kam es zu einer deutlichen Verzögerung bei der Einhaltung des medizinischen Therapieprotokolls?', 'Kam es zu lebe

,Participant ID,Survey Identifier,Survey Timestamp,Datum,CTCAE-Bewertung,"Was war das AE? (Schmerz, Schürfwunde etc.)",Hat jemand mit Expertise die Situation geprüft?,Diese Person hat ein 'OK' zum Weitermachen gegeben.,Wurde etwas Strukturelles verändert?,Was?,...,Welche motorische Hauptform dominierte die Bewegungseinheit ? (choice=Schnelligkeit).2,In welcher Phase der geplanten Sporteinheit trat das AE auf?.2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Gut).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Durchschnittlich).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Mäßig).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Weiß nicht).2,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Anmerkung).2,"Glaubst Du, dass das AE mit dem Sport assoziiert ist?",Anmerkung.2,Complete?.2
0,1,NaN,nan,nan,NaN,nan,nan,nan,nan,nan,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
1,2,NaN,2022-10-13 12:26:32,nan,1.0,Schürfwunde an der Wange,nan,Ja,No,nan,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete
2,3,NaN,2022-10-13 13:41:16,nan,2.0,strake Übelkeit,nan,nan,No,nan,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,Incomplete


ValueError: Cannot accept list of column references or list of columns for both `x` and `y`.